In [19]:
import numpy as np
from get_border_cases import get_border_cases
import matplotlib as mpl
import matplotlib.colors as mcolors
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [20]:
n = 17
N = n - 1
inner_n = n - 2
border_cases = get_border_cases(N)
params = np.ones(inner_n ** 2)

In [21]:
def find_max(func, game):
    if func(0, game) < func(1, game):
        v = func(0, game)
        p = 0
    else:
        v = func(1, game)
        p = 1
    return v, p

def get_win(p, game):
    return (p / 2) * (game[0, 0] + game[1, 0]) + (1 - p) / 2 * (game[0, 1] + game[1, 1])

'''
    Computes the value of a game.
    game is 2x2 matrix.
'''
def get_value(game):
    v, q = find_max(get_win, game)
    return v, q

'''
    Converts global index to inner index.
    index - index to convert.
    inner_n - inner dimension (without borders).
    global_n - outer dimension (with borders).
'''
def inner_n_to_global_N(index, inner_n, global_n):
    row = index // inner_n
    col = index % inner_n
    global_index = (row + 1) * global_n + (col + 1)
    return global_index

def global_N_to_inner_n(index, inner_n, global_n):
    row = index // global_n
    col = index % global_n

    if row < 1 or col < 1 or row > global_n - 2 or col > global_n - 2:
        raise ValueError("Index should match an inner node.")
    inner_index = (row - 1) * inner_n + (col - 1)
    return inner_index

# q == 0.5
def get_game(index, w, epsilon):
    game = np.zeros((2, 2))
    game[0, 0] = compute_a11(index, w, epsilon)
    game[0, 1] = compute_a12(index, w, epsilon)
    game[1, 0] = compute_a21(index, w, epsilon)
    game[1, 1] = compute_a22(index, w, epsilon)
    return game

# Up
def compute_a11(index, w, epsilon):
    if (index - n) in border_cases:
        a11 = 1
    else:
        a11 = w[global_N_to_inner_n(index - n, inner_n, n)] + 1 + epsilon
    return a11
    
# Down
def compute_a21(index, w, epsilon):
    if (index + n) in border_cases:
        a12 = 1
    else:
        a12 = w[global_N_to_inner_n(index + n, inner_n, n)] + 1 + epsilon
    return a12

# Right
def compute_a12(index, w, epsilon):
    if (index + 1) in border_cases:
        a21 = 1
    else:
        a21 = w[global_N_to_inner_n(index + 1, inner_n, n)] + 1 + epsilon
    return a21

# Left
def compute_a22(index, w, epsilon):
    if (index - 1) in border_cases:
        a22 = 1
    else:
        a22 = w[global_N_to_inner_n(index - 1, inner_n, n)] + 1 + epsilon
    return a22
    

'''
    Number of values in w should match number of equations.
'''
def prepare_equations(w, epsilon):
    eqs = np.zeros(len(w))
    p1s = []
    q1s = []
    for i in range(len(w)):
        index = inner_n_to_global_N(i, inner_n, n)
        game_mx = get_game(index, w, epsilon)
        
        v, q1 = get_value(game_mx)
        p1 = 0.5

        p1s.append(p1)
        q1s.append(q1)
        eqs[i] = w[i] - v

    return tuple(eqs)

In [ ]:
# first launch first line and then uncomment others

from scipy.optimize import fsolve

file = open("D:\RW\\bvp\\epsilon_values.txt", "w")
epsilon_values = np.arange(0.0, 0.151, 0.002)
for eps in epsilon_values:
    file.write(f"{eps} ")
file.close()
w_new_list = []
for epsilon in epsilon_values:
    message = ""
    while (message != "The solution converged."):
        starting_params = np.random.random(inner_n ** 2) * (inner_n - 1) ** 2
        w_new, _, _, message  = fsolve(lambda w: prepare_equations(w, epsilon), tuple(starting_params), full_output=True)
        #print(message)
        w_new_list.append(w_new)
    starting_params

In [23]:
step = 0
epsilon_values = np.arange(0.0, 0.151, 0.002)
p1s = [[] for _ in range(len(epsilon_values))]
q1s = [[] for _ in range(len(epsilon_values))]
vs = [[] for _ in range(len(epsilon_values))]
for epsilon in epsilon_values:
    for i in range(len(w_new)):
        index = inner_n_to_global_N(i, inner_n, n)
        game_mx = get_game(index, w_new_list[step], epsilon)
        
        v, q1 = get_value(game_mx)
        p1 = 0.5
            
        p1s[step].append(p1)
        q1s[step].append(q1)
        vs[step].append(v)
    step +=1

In [ ]:
epsilon_values = np.arange(0.0, 0.151, 0.002)
step = 0
file = open("D:\RW\\bvp\\try_5\\absorption_time.txt", "w")
for epsilon in epsilon_values:
    fig, ax = plt.subplots(1, 1, figsize=(15, 15))
    a = ax.imshow(np.reshape(vs[step], newshape=(inner_n, inner_n)))
    fig.colorbar(a)
    for (j, i), label in np.ndenumerate(np.reshape(vs[step], newshape=(inner_n, inner_n))):
        #print_str_1 = "↑→" if np.isclose(label, 1) else ""
        #print_str_2 = "↓←" if np.isclose(label, 0) else ""
        if i == 7 and j == 7:
            file.write(f"{label:.2f} ")
        ax.text(i, j, f"{label:.2f} ", ha='center', va='center')
        ax.text(i, j, f"{label:.2f} ", ha='center', va='center')
        #print(label)
    ax.set_title("Mean absorption time")
    str_ = "bvp_strategy_" + str(epsilon) + ".png"
    fig.savefig("D:\RW\\bvp\\try_5\\1\\" + str_)
    plt.close()
    step += 1
file.close()

In [25]:
border_color_3 = np.array([204, 153, 255]) / 255.0
border_color_2 = np.array([178, 102, 255]) / 255.0
border_color_1 = np.array([141, 29, 255]) / 255.0
white_color = np.array([255, 255, 255]) / 255.0
center_color_1 = np.array([255, 202, 26]) / 255.0
center_color_2 = np.array([255, 219, 102]) / 255.0
center_color_3 = np.array([255, 231, 153]) / 255.0
rwgame_cmap = mcolors.LinearSegmentedColormap.from_list(
        'rwgame', [border_color_1, border_color_2, border_color_3, white_color, center_color_3, center_color_2, center_color_1])

In [ ]:
epsilon_values = np.arange(0.0, 0.151, 0.002)
step = 0
for epsilon in epsilon_values:
    fig, ax = plt.subplots(1, 1, figsize=(15, 15))
    a = ax.imshow(np.reshape(p1s[step], newshape=(inner_n, inner_n)))
    fig.colorbar(a)

    for (j, i), label in np.ndenumerate(np.reshape(p1s[step], newshape=(inner_n, inner_n))):
        print_str_1 = "↑→" if np.isclose(label, 1) else ""
        print_str_2 = "↓←" if np.isclose(label, 0) else ""
        plt.text(i, j, f"{label:.2f} " + print_str_1 + print_str_2, ha='center', va='center')
        plt.text(i, j, f"{label:.2f} " + print_str_1 + print_str_2, ha='center', va='center')
    ax.set_title("Вероятность игрока \"за центр\" выбрать 1 стратегию.")
    str_ = "bvp_strategy_" + str(epsilon) + ".png"
    fig.savefig("D:\RW\\bvp\\try_5\\2\\" + str_)
    plt.close()
    #fig.savefig("G:\Studies\Masters\img\\bvp_strategy.png")
    step += 1

In [ ]:
epsilon_values = np.arange(0.0, 0.151, 0.002)
step = 0
for epsilon in epsilon_values:
        font = {'family' : 'normal',
                'size'   : 5}

        mpl.rc('font', **font)

        fig, ax = plt.subplots(1, 1, dpi=300)
        ax.invert_yaxis()
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        a = ax.imshow(np.reshape(q1s[step], newshape=(inner_n, inner_n)), cmap=rwgame_cmap)


        for (j, i), label in np.ndenumerate(np.reshape(q1s[step], newshape=(inner_n, inner_n))):
                print_str_1 = "\n↑↓" if np.isclose(label, 1) else ""
                print_str_2 = "\n→←" if np.isclose(label, 0) else ""
                plt.text(i, j, f"{label:.2f} " + print_str_1 + print_str_2, ha='center', va='center')
                plt.text(i, j, f"{label:.2f} " + print_str_1 + print_str_2, ha='center', va='center')

        font = {'family' : 'normal',
                'size'   : 8}
        mpl.rc('font', **font)

        cbar = fig.colorbar(a)
        cbar.ax.text(4, 1, '↑\n↓', ha='center', va='center')
        cbar.ax.text(2, 0.03, '0', ha='center', va='center')
        cbar.ax.text(4, 0.03, '→\n←', ha='center', va='center')

        ax.set_title("Вероятность игрока \"за границу\" выбрать 1 стратегию.")
        str_ = "bvp_2_strategy_" + str(epsilon) + ".png"
        fig.savefig("D:\RW\\bvp\\try_5\\3\\" + str_)
        plt.close()
        #fig.savefig("G:\Studies\Masters\img\\bvp_2_strategy.png")
        step += 1
